In [1]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(42)

ROWS = 12000

data = {
    # Demographics
    "age": np.random.randint(25, 85, ROWS),
    "sex": np.random.binomial(1, 0.55, ROWS),  # 55% male
    
    # Clinical measurements
    "cholesterol": np.random.normal(240, 50, ROWS),
    "resting_bp": np.random.normal(135, 20, ROWS),
    "max_heart_rate": np.random.normal(150, 25, ROWS),
    
    # Blood indicators
    "fasting_blood_sugar": np.random.binomial(1, 0.25, ROWS),
    "blood_sugar_level": np.random.normal(110, 30, ROWS),
    
    # Symptoms
    "exercise_angina": np.random.binomial(1, 0.3, ROWS),
    "oldpeak": np.abs(np.random.normal(1.5, 1.2, ROWS)),
    
    # Lifestyle / history
    "smoking": np.random.binomial(1, 0.35, ROWS),
    "obesity": np.random.binomial(1, 0.4, ROWS),
}

df = pd.DataFrame(data)

# --- Introduce medical realism ---
df["cholesterol"] = df["cholesterol"].clip(120, 450)
df["resting_bp"] = df["resting_bp"].clip(80, 200)
df["max_heart_rate"] = df["max_heart_rate"].clip(60, 210)
df["blood_sugar_level"] = df["blood_sugar_level"].clip(60, 300)
df["oldpeak"] = df["oldpeak"].clip(0, 6)

# --- Disease probability logic (REALISTIC) ---
risk_score = (
    0.03 * df["age"] +
    0.02 * df["cholesterol"] +
    0.025 * df["resting_bp"] +
    0.04 * df["fasting_blood_sugar"] * 100 +
    0.05 * df["exercise_angina"] * 100 +
    0.04 * df["smoking"] * 100 +
    0.04 * df["obesity"] * 100 +
    0.03 * df["oldpeak"] * 10 -
    0.02 * df["max_heart_rate"]
)

probability = 1 / (1 + np.exp(-0.03 * (risk_score - risk_score.mean())))

df["target"] = (probability > 0.65).astype(int)

# --- Enforce class imbalance (industry realistic) ---
# ~80% healthy, ~20% disease
df.loc[df.sample(frac=0.25, random_state=42).index, "target"] = 0

# --- Inject missing values (noise) ---
for col in ["cholesterol", "resting_bp", "blood_sugar_level"]:
    df.loc[df.sample(frac=0.07, random_state=42).index, col] = np.nan

# --- Inject impossible values (dirty hospital data) ---
df.loc[df.sample(frac=0.01, random_state=42).index, "cholesterol"] = 0
df.loc[df.sample(frac=0.01, random_state=24).index, "resting_bp"] = 0

print(df.head())
print("\nShape:", df.shape)
print("\nTarget distribution:\n", df["target"].value_counts(normalize=True))




risk = (
    (df["cholesterol"] > 240).astype(int) +
    (df["resting_bp"] > 140).astype(int) +
    df["exercise_angina"] +
    df["smoking"] +
    df["obesity"] +
    (df["age"] > 55).astype(int)
)

df["target"] = (risk >= 3).astype(int)


   age  sex  cholesterol  resting_bp  max_heart_rate  fasting_blood_sugar  \
0   63    1   158.236706  171.826261      181.703149                    0   
1   76    1   307.802317  143.901204      126.261537                    0   
2   53    1   292.632469  159.715218      150.652781                    0   
3   39    1   216.365781  147.477747      191.136913                    0   
4   67    0   253.091766  126.739457      140.254895                    0   

   blood_sugar_level  exercise_angina   oldpeak  smoking  obesity  target  
0         177.554990                0  2.217908        1        1       0  
1          83.504048                1  1.480675        0        0       0  
2         112.736859                0  0.072005        1        0       0  
3         149.440470                0  2.377410        0        0       0  
4         109.788175                1  2.550232        0        0       0  

Shape: (12000, 12)

Target distribution:
 target
0    1.0
Name: proportion, dtyp

In [4]:
df

,age,sex,cholesterol,resting_bp,max_heart_rate,fasting_blood_sugar,blood_sugar_level,exercise_angina,oldpeak,smoking,obesity,target
0,63,1,158.236706,171.826261,181.703149,0,177.554990,0,2.217908,1,1,1
1,76,1,307.802317,143.901204,126.261537,0,83.504048,1,1.480675,0,0,1
2,53,1,292.632469,159.715218,150.652781,0,112.736859,0,0.072005,1,0,1
3,39,1,216.365781,147.477747,191.136913,0,149.440470,0,2.377410,0,0,0
4,67,0,253.091766,126.739457,140.254895,0,109.788175,1,2.550232,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,57,1,NaN,NaN,166.054175,0,NaN,0,0.757018,1,1,1
11996,70,0,178.429561,123.167288,149.343325,1,107.229654,0,2.060941,0,0,0
11997,41,1,218.337595,152.408872,169.674321,0,150.730573,0,0.663092,1,0,0
11998,66,1,NaN,NaN,182.172033,0,NaN,0,0.956816,1,0,0


In [9]:
x = df.iloc[:,0 : -1]
y = df.iloc[:,-1]

In [11]:
from sklearn.model_selection import train_test_split

In [61]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [62]:
x_train[x_train['cholesterol'] == 0].shape

(101, 11)

In [69]:
# x_train['cholesterol'] = x_train['cholesterol'].replace(0, np.nan)
x_train['cholesterol']  = np.where(x_train['cholesterol']==0, np.nan, x_train['cholesterol'])

In [70]:
x_train[x_train['cholesterol'] == 0].shape


(0, 11)

In [75]:
x_train['cholesterol'].median()
# x_train[x_train['cholesterol'] == 0].shape

240.2581406457031

In [76]:
x_train['cholesterol'].fillna(x_train['cholesterol'].median(),inplace=True)

In [77]:
df.head(1)

,age,sex,cholesterol,resting_bp,max_heart_rate,fasting_blood_sugar,blood_sugar_level,exercise_angina,oldpeak,smoking,obesity,target
0,63,1,158.236706,171.826261,181.703149,0,177.55499,0,2.217908,1,1,1


In [82]:
df[df['resting_bp']  > 140].shape

(4393, 12)

In [83]:
x_train['alarming_resting_bp'] = np.where(x_train['resting_bp'] > 140, 1, 0)

In [84]:
x_train

,age,sex,cholesterol,resting_bp,max_heart_rate,fasting_blood_sugar,blood_sugar_level,exercise_angina,oldpeak,smoking,obesity,alarming_resting_bp
2918,32,1,183.824483,163.876014,163.380884,0,125.002841,0,0.007228,0,1,1
3581,61,1,224.779953,129.254869,159.122431,1,103.608059,1,2.678471,0,1,0
3303,72,1,201.130562,134.928776,149.302883,0,98.742599,0,1.013916,1,1,0
3583,28,0,281.782863,125.616363,154.987525,1,128.666454,1,0.909779,1,1,0
7841,69,0,240.258141,NaN,120.916711,0,NaN,1,1.462704,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8088,46,1,265.597810,117.264973,202.949420,0,129.776503,1,0.578108,0,1,0
10107,55,0,274.418030,124.656251,163.069303,0,75.255892,0,2.474240,1,1,0
4631,45,1,234.903781,134.002015,118.367214,0,60.000000,0,2.330620,0,0,0
1738,30,1,231.568355,152.587566,108.581073,0,60.000000,1,1.969861,0,0,1
